In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [149]:
cities = ['ABZ', 'ALE', 'AMA', 'AMM', 'ASP', 'BEI', 'BOT', 'BSL', 'DEL', 'EGB',
          'HAD', 'HEL', 'HPB', 'HYY', 'KCE', 'KPZ', 'MAR', 'MHD', 'MLP', 'MUK', 
          'NAN', 'NEU', 'POV', 'SAO', 'SCH', 'SGP', 'UAE', 'VAR', 'VIE', 'WAL', 
          'ZOT']

In [150]:
def combine_datasets(folder):
    full_df = []

    for c in cities:
        df = pd.read_csv('data/'+folder+'/'+c+'.csv')
        df['station'] = c
        full_df.append(df)

    full_df = pd.concat(full_df)
    full_df = full_df.reset_index(drop=True)
    full_df['id'] = [full_df.station[i] + '-' + str(full_df.date[i]) for i in range(full_df.shape[0])]
    
    return full_df

In [166]:
aerosols_df = combine_datasets('aerosols')
aerosols_df = aerosols_df[['id', 'latitude', 'longitude',
                           'aermr01', 'aermr02', 'aermr03','aermr04', 'aermr05',
                           'aermr06', 'aermr07', 'aermr08',  'aermr09', 'aermr10']]
# exclude 'aermr11' for now so that there is still data for AMA

aerosols_df.to_csv('data/aerosols_data.csv', index=False)

In [141]:
atmospheric_df = combine_datasets('atmospheric')
atmospheric_df = atmospheric_df[['id', 'd2m', 't2m']]

atmospheric_df.to_csv('data/atomospheric_data.csv', index=False)

In [142]:
boundary_layer_height_df = combine_datasets('boundary_layer_height')
boundary_layer_height_df = boundary_layer_height_df[['id', 'blh']]

boundary_layer_height_df.to_csv('data/boundary_layer_height_data.csv', index=False)

In [143]:
gases_df = combine_datasets('gases')
gases_df = gases_df[['id', 'co', 'c5h8', 'no2', 'no', 'so2']]

gases_df.to_csv('data/gases_data.csv', index=False)

In [144]:
slow_access_df = combine_datasets('slow_access')
slow_access_df = slow_access_df[['id', 'nh3', 'crwc', 'c10h16']]
slow_access_df.to_csv('data/slow_access_data.csv', index=False)

In [134]:
n100_df = []

for c in cities:
    df = pd.read_table('data/N100_proxy/'+c+'_N100.dat', sep='\s+', 
                       names=['year', 'month', 'day', 'hour', 'minute', 'n100'])
    
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    
    # AMA and UAE are in UTC, so the times of the measurements need to be adjusted to local time
    if (c == 'AMA'):
        df['time'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.time -= timedelta(hours=4)
        df = df.drop(columns='time')
    elif (c == 'UAE'):
        df['time'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.time += timedelta(hours=4)
        df = df.drop(columns='time')
    
    df = df.groupby('date', as_index=False).mean()
    df.date = df.date.dt.date
    df['station'] = c
    n100_df.append(df)

n100_df = pd.concat(n100_df)
n100_df = n100_df.reset_index(drop=True)
n100_df['id'] = [n100_df.station[i] + '-' + str(n100_df.date[i]) for i in range(n100_df.shape[0])]
n100_df = n100_df[['id', 'station', 'date', 'n100']]
n100_df.to_csv('data/n100_data.csv', index=False)

In [172]:
# combining the data sets into one data frame
data = n100_df.merge(aerosols_df, on='id')
data = data.merge(atmospheric_df, on='id')
data = data.merge(boundary_layer_height_df, on='id')
data = data.merge(gases_df, on='id')
data = data.merge(slow_access_df, on='id')

data.date = pd.to_datetime(data.date)
data = data.dropna(axis=0)
data.to_csv('data/full_data.csv', index=False)